In [17]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=False,      # For video
    model_complexity=1,           # 0=Light, 1=Full, 2=Heavy
    smooth_landmarks=True,        # Temporal smoothing
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

In [18]:
def detect_fall_in_video(video_path, output_path="output_with_fall_detection.mp4"):
    # Open video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties for output
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Create VideoWriter to save output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Variables for fall detection
    prev_y = 0
    prev_time = time.time()
    velocity = 0
    fall_detected = False
    fall_start_frame = 0
    fall_counter = 0
    
    # Process each frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert BGR to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
        
        # Process with MediaPipe
        results = pose.process(image_rgb)
        
        # Convert back to BGR for OpenCV
        image_rgb.flags.writeable = True
        annotated_frame = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        
        # FALL DETECTION LOGIC
        if results.pose_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(
                annotated_frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS
            )
            
            # Get key landmarks
            landmarks = results.pose_landmarks.landmark
            
            # Use mid-hip (landmarks 23 & 24 average) as reference
            hip_y = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y + 
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y) / 2
            
            # Calculate normalized height (0 to 1, where 1 is full height)
            # Shoulder (landmark 11 & 12) to ankle (landmark 27 & 28) distance
            shoulder_y = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y + 
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y) / 2
            
            ankle_y = (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y + 
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y) / 2
            
            body_height_ratio = abs(shoulder_y - ankle_y)
            
            # Calculate velocity
            current_time = time.time()
            time_diff = current_time - prev_time
            if time_diff > 0:
                velocity = (hip_y - prev_y) / time_diff
            
            # Update previous values
            prev_y = hip_y
            prev_time = current_time
            
            # FALL DETECTION CONDITIONS
            # Condition 1: Sudden downward velocity
            # Condition 2: Body becomes more horizontal (height ratio decreases)
            # Condition 3: Person is on the ground (hip close to ankle)
            
            if (velocity > 0.5 and              # Fast downward movement
                body_height_ratio < 0.3 and     # Body is compressed/horizontal
                (hip_y - ankle_y) < 0.1):       # Hip is close to ground
                
                if not fall_detected:
                    fall_detected = True
                    fall_start_frame = fall_counter
                    print(f"FALL DETECTED at frame {fall_counter}")
                
                # Draw warning on frame
                cv2.putText(annotated_frame, "FALL DETECTED!", (50, 50),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.rectangle(annotated_frame, (0, 0), (width, height), 
                             (0, 0, 255), 10)  # Red border
            else:
                if fall_detected and (fall_counter - fall_start_frame > 30):
                    fall_detected = False
            
            # Display metrics on frame (for debugging)
            cv2.putText(annotated_frame, f"Velocity: {velocity:.3f}", (50, 100),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(annotated_frame, f"Height Ratio: {body_height_ratio:.3f}", 
                       (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Write frame to output
        out.write(annotated_frame)
        fall_counter += 1
        
        # Display the frame (optional)
        cv2.imshow('Baby Fall Detection', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    pose.close()
    print(f"Processing complete. Output saved to {output_path}")

In [19]:
if __name__ == "__main__":
    # Replace with your video file path
    input_video = "Fall.mp4"
    
    # Check if video exists
    import os
    if not os.path.exists(input_video):
        print(f"Error: Video file '{input_video}' not found.")
        print("Please provide a valid video path.")
    else:
        detect_fall_in_video(input_video)

FALL DETECTED at frame 0
FALL DETECTED at frame 99
Processing complete. Output saved to output_with_fall_detection.mp4
